In [1]:
# Say we need to develop a smart attendance system using AI.
# Create a system that accepts two IP cameras for Login/Logout doors, each camera will detect the face  
# and search in faces DB,then save when attending and when leaving!

# Assumption 1 - A DB of the group's faces images. Say we have N people in the group.
# Assumption 2 - There is no other exit to the classroomm meaning going in the class will show you once on 
# the login camera and going out will show you once in the logout camera

# The way i decided to implement this is to train a CNN the receives the input from the cameras, 
# The CNN will be trained on detecting faces and classifing them to one of the faces that appear in the DB
# The CNN output layer should use softmax with N nodes to give probabillity which one of the faces is the face 
# in the picture recieved by the camera.

# We will use the chokepoint dataset - https://arma.sourceforge.net/chokepoint
# With PiE being login camera and PiL being logout camera.
from tensorflow import keras
import tensorflow as tf



In [2]:
pathsToDB = [
    "FaceDetectionDB/P1E_S1/P1E_S1_C1",
    "FaceDetectionDB/P1E_S2/P1E_S2_C1",
    "FaceDetectionDB/P1E_S3/P1E_S3_C1",
    "FaceDetectionDB/P1E_S4/P1E_S4_C1",
#     "FaceDetectionDB/P1L_S1/P1L_S1_C1",
#     "FaceDetectionDB/P1L_S2/P1L_S2_C1",
#     "FaceDetectionDB/P1L_S3/P1L_S3_C1",
#     "FaceDetectionDB/P1L_S4/P1L_S4_C1",
]

In [3]:
from PIL import Image
import numpy
import os
X = []
for path in pathsToDB:
    print(path)
    imgs = os.listdir(path)
    imgs.sort()
    for img in imgs:
        if img[-1] != "g":
            continue
        p = path + "/" + img
        imgFile = Image.open(p)        
        X.append(numpy.array(imgFile))


FaceDetectionDB/P1E_S1/P1E_S1_C1
FaceDetectionDB/P1E_S2/P1E_S2_C1
FaceDetectionDB/P1E_S3/P1E_S3_C1
FaceDetectionDB/P1E_S4/P1E_S4_C1


In [5]:
#Create DB
# This DB was taken from https://arma.sourceforge.net/chokepoint/#download
from xml.dom import minidom
import numpy as np


# parse xml file by name

def containsPerson(frame):
    return 1 if frame.childNodes else 0


def returnLabel(frame): return int(frame.childNodes[1].attributes['id'].value) if frame.childNodes else 0


def getLabelsForFrames(xmlFilePath):
    file = minidom.parse(xmlFilePath)
    frames = file.getElementsByTagName('frame')
    y_contains_person = np.array([containsPerson(frame) for frame in frames])
    y_label = np.array([returnLabel(frame) for frame in frames])
    return y_contains_person, y_label


yE1s1_contains, yE1s1_label = getLabelsForFrames("groundtruth/P1E_S1_C1.xml")
yE1s2_contains, yE1s2_label = getLabelsForFrames("groundtruth/P1E_S2_C1.xml")
yE1s3_contains, yE1s3_label = getLabelsForFrames("groundtruth/P1E_S3_C1.xml")
yE1s4_contains, yE1s4_label = getLabelsForFrames("groundtruth/P1E_S4_C1.xml")
# yEContains = np.concatenate((yE1s1_contains,yE1s2_contains,yE1s3_contains,yE1s4_contains))
yElabels = np.concatenate((yE1s1_label, yE1s2_label, yE1s3_label,yE1s4_label))
# yL1s1_contains, yL1s1_label = getLabelsForFrames("groundtruth/P1L_S1_C1.xml")
# yL1s2_contains, yL1s2_label = getLabelsForFrames("groundtruth/P1L_S2_C1.xml")
# yL1s3_contains, yL1s3_label = getLabelsForFrames("groundtruth/P1L_S3_C1.xml")
# yL1s4_contains, yL1s4_label = getLabelsForFrames("groundtruth/P1L_S4_C1.xml")
# yLContains = np.concatenate((yL1s1_contains, yL1s2_contains, yL1s3_contains, yL1s4_contains))
# yLlabels = np.concatenate((yL1s1_label, yL1s2_label, yL1s3_label, yL1s4_label))


In [7]:
import numpy
import tensorflow as tf
# y = np.concatenate((yElabels,yLlabels))
y=yElabels

In [8]:
len(y)

9487

In [9]:
len(X)

9487

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [11]:
len(X)

9487

In [12]:
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)


In [13]:
def preprocess(image):
    resized_image = tf.image.resize(image, [299, 299])
    final_image = keras.applications.xception.preprocess_input(resized_image) 
    return final_image
X_val_1 = preprocess(X_val)

2023-02-21 11:21:21.656866: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
X_train_1 = preprocess(X_train)

In [15]:
X_test_1 = preprocess(X_test)
n_classes = 27
batch_size = 32

In [16]:
type(X_train_1)

tensorflow.python.framework.ops.EagerTensor

In [17]:
X_train_1.shape

TensorShape([6071, 299, 299, 3])

In [18]:
base_model = keras.applications.xception.Xception(weights="imagenet",include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation="softmax")(avg)
model = keras.models.Model(inputs=base_model.input, outputs=output)

In [19]:
for layer in base_model.layers: 
    layer.trainable = False

In [20]:
X_train_1.shape

TensorShape([6071, 299, 299, 3])

In [21]:
len(X_train_1)

6071

In [22]:
len(y_train)

6071

In [23]:
type(y_train)

numpy.ndarray

In [24]:
max(y_train)

27

In [25]:
f = lambda x: x-1 if x>0 else x
y_train_1 = map(f,y_train)
y_test_1 = map(f,y_test)
y_val_1 = map(f,y_val)
y_train_1 = np.array(list(y_train_1))
y_test_1 = np.array(list(y_test_1))
y_val_1 = np.array(list(y_val_1))

In [29]:
base_model = keras.applications.xception.Xception(weights="imagenet",include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation="softmax")(avg)
model = keras.models.Model(inputs=base_model.input, outputs=output)

for layer in base_model.layers: 
    layer.trainable = False

optimizer = keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(X_train_1,y=y_train_1,batch_size=batch_size,
                    steps_per_epoch=int(0.75 * len(X_train_1) / batch_size),
                    validation_data=(X_val_1, y_val_1) ,
                    validation_steps=int(0.15 * len(X_val_1) / batch_size),
                    epochs=2)


Epoch 1/2
142/142 [==============================] - 338s 2s/step - loss: 0.8038 - accuracy: 0.7951 - val_loss: 0.2993 - val_accuracy: 0.9107
Epoch 2/2
142/142 [==============================] - 339s 2s/step - loss: 0.2384 - accuracy: 0.9405 - val_loss: 0.2196 - val_accuracy: 0.9509


In [32]:
for layer in base_model.layers: 
    layer.trainable = True

In [34]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)
optimizer = keras.optimizers.SGD(lr=0.05, momentum=0.9, decay=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history2 = model.fit(X_train_1,y=y_train_1,batch_size=batch_size,
                    steps_per_epoch=int(0.75 * len(X_train_1) / batch_size),
                    validation_data=(X_val_1, y_val_1),
                    validation_steps=int(0.15 * len(X_val_1) / batch_size),
                    epochs=3,callbacks=[early_stopping_cb])

Epoch 1/3
142/142 [==============================] - 1567s 11s/step - loss: 0.2564 - accuracy: 0.9349 - val_loss: 0.8552 - val_accuracy: 0.7679
Epoch 2/3
142/142 [==============================] - 1546s 11s/step - loss: 0.0835 - accuracy: 0.9749 - val_loss: 0.0517 - val_accuracy: 0.9777
Epoch 3/3
142/142 [==============================] - 1502s 11s/step - loss: 0.0429 - accuracy: 0.9874 - val_loss: 0.0656 - val_accuracy: 0.9777


In [36]:
model.save("my_keras_model.h5")

In [58]:
y_test = np.array(list(y_test))

In [63]:
score = model.evaluate(X_test_1, y_test_1, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

60/60 [==============================] - 137s 2s/step - loss: 0.0653 - accuracy: 0.9821
Test loss: 0.06526342034339905
Test accuracy: 0.9820864200592041


In [68]:
print(np.argmax(model.predict(np.array([X_test_1[0],]))))
print(y_test_1[0])

1/1 [==============================] - 0s 133ms/step
19
19
